In [26]:
import pandas as pd
import numpy as np
from sklearn import linear_model
%matplotlib inline

In [27]:
data = pd.read_excel('HealthViz County Dataset 6.19.17.xlsx',skiprows=0, header=1, index_col=0)
data.index.name=None
#wgt = np.array(data['Population (residents), 2011-2015'])
data.drop(data.columns[[0,1]], axis=1, inplace=True)

In [28]:
#import pandas as pd
#df = pd.read_csv('HealthViz County Dataset 6 21 17.csv', skiprows=0, header=1, index_col=0,encoding = "ISO-8859-1")
#df.index.name=None

In [29]:
data.isnull().sum()

Median household income, 2011-2015                                                         3
Per capita income, 2011-2015                                                               2
Gini index of income inequality, 2015                                                   2390
Food stamps (SNAP) (% of households), 2011-2015                                            2
Poverty rate (% of residents), 2011-2015                                                   2
Median property tax ($), 2011-2015                                                        36
Rent-burdened (% of renter-occupied housing units), 2011-2015                              2
Median gross rent, 2011-2015                                                               3
Median home value, 2011-2015                                                               4
Occupied (% of housing units), 2011-2015                                                   2
Medicare flu vaccination rate (% of beneficiaries), 2015              

In [30]:
# Using linear regression to fill out the missing values in certain columns based on the rest of columns which have
# the full values. The fullfilled colunms will be considered as columns with full values to make prediction in the 
# later run.

# Step 0: Have the original dataset
# Step 1: Get the dataframe with all the columns with full values (Xs) and the colume with least missing value (Y)
# Step 2: Split the dataframe into trainning (Xs and Y with values) and predict datasets (Xs and Y with missing values)
# Step 3: Run the regression with the trainning set and get the predictions for the missing Y values in the predict set
# Step 4: The new dataframe has been created with the fullfillment of the Y
# Step 5: Merge the new dataframe with the other potential Ys to replace the orginal dataset to iterate the above steps
# until all the Ys have been fullfilled

def preprocess_df(data):
    '''
    Given a (controlled variables) dataframe, output a dataframe with the full-value columns (Xs) and the column with\
    least missing values (Y)
    
        data: dataframe to be dealt with
        Return: dataframe with only used Xs, full dataframe with used Xs and the Y, dataframe with the other Ys
    '''
    df_xs = data.dropna(axis=1, how='any')
    l_ys = data.columns[data.isnull().any()].tolist()
    len_l_ys = len(l_ys)
    i_min = 0
    df_y = pd.DataFrame(data[l_ys[i_min]])
    n_min = df_y.isnull().values.ravel().sum()
    for i in range(1, len_l_ys):
        df_y = pd.DataFrame(data[l_ys[i]])
        n = df_y.isnull().values.ravel().sum()
        if n < n_min:
            n_min = n
            i_min = i
    label_y = l_ys[i_min]
    df_y = pd.DataFrame(data[l_ys[i_min]])
    df = pd.merge(df_y, df_xs, right_index=True, left_index=True)
    df_ys = data[data.columns[data.isnull().any()]].drop(label_y, 1)
    return df_xs, df_ys, df


def train_predict_split(df):
    '''
    Given the full dataset, split it into train and predict datasets based on the missing value of the Y
    The first column of the dataset is the Y
    '''
    labels = list(df.columns)
    mask = df[labels[0]].isnull()
    df_predict = df[mask]
    df_train = df[~mask]
    y_train = df_train.ix[:, 0:1]
    x_train = df_train.ix[:, 1:]
    x_predict = df_predict.ix[:, 1:]
    y_predict = df_predict.ix[:, 0:1]
    return x_train, y_train, x_predict, df_predict, df_train
    
def new_predict_df(x_train, y_train, x_predict, df_predict, df_train):
    '''
    Filling the missing values and return a new Xs and the Y dataframe
    '''
    lm = linear_model.LinearRegression()
    model = lm.fit(x_train,y_train)
    y_pred = lm.predict(x_predict)
    l = []
    d = {}
    for i in y_pred.tolist():
        l = l + i
    se = pd.Series(l)
    df_predict['prediction'] = se.values
    df_predict[y_train.columns.tolist()[0]].fillna(value=df_predict['prediction'], inplace=True)
    df_predict = df_predict.drop('prediction', 1)
    df = df_train.append(df_predict)
    return df

def get_new(df):
    '''
    Given a data set, get the new precit data set with the predict values
    '''
    x_train, y_train, x_predict, df_predict, df_train = train_predict_split(df)
    df_new = new_predict_df(x_train, y_train, x_predict, df_predict, df_train)
    return df_new

In [31]:
num_null = len(data.columns[data.isnull().any()].tolist())
for i in range(0, num_null):
    df_xs, df_ys, df = preprocess_df(data)
    df_new = get_new(df)
    data = pd.merge(df_ys, df_new, right_index=True, left_index=True)    

/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/admin/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:

In [32]:
data.isnull().sum()

Gini index of income inequality, 2015                                                   0
Diagnosed diabetes (% of adults), 2006-2012                                             0
High blood pressure (% of adults), 2006-2012                                            0
Obesity (% of adults), 2006-2012                                                        0
Years of potential life lost (YPLL) (life-years lost per 100,000), 2012-2014            0
Mortality rate, all causes (deaths per 100,000), 2008-2014                              0
Medicare emergency department visit rate (visits per 1,000 beneficiaries), 2015         0
Medicare FFS per capita costs ($ per beneficiary), 2015                                 0
Septicemia (sepsis) mortality (deaths per 100,000), 2008-2014                           0
Influenza and pneumonia mortality (deaths per 100,000), 2008-2014                       0
Diabetes mortality (deaths per 100,000), 2008-2014                                      0
Medicare i